# Tema: Avaliação de jogos
- The Last Of Us Remastered
- Duke Nukem Forever
- God of War: Ragnarok
- No Man's Sky
- Biomutant
- CyberPunk 2077

# Objetivo 
- Desenvolver um crawler para coletar do site de avaliações de filmes e jogos(Metacritic) e construir um modelo de analise de sentimento

## Criação do Crawler

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def get_name(vreview):
    try:
        vname = vreview.find('div', class_='name').find('a').text
    except:
        vname = vreview.find('div', class_='review_body').find('span', class_='blurb blurb_expanded')
    return vname

def get_rating(vreview):
    try:
        vrating = vreview.find('div', class_='review_grade').text.strip('\n')
    except:
        vrating = None
    return vrating

def get_date(vreview):
    return vreview.find('div', class_='date').text

def get_coment(vreview):
    try:
        vcomment = vreview.find('span', class_='blurb blurb_expanded').text
    except: 
        vcomment = vreview.find('div', class_='review_body').find('span', class_='blurb blurb_expanded')
    return vcomment

def get_title(vurl):

    bs_title = BeautifulSoup(response.text, 'html.parser')
    vtitle = bs_title.find('div', class_='product_title').find('a').text.strip('\n')

    return vtitle

def get_last_pages(vurl, pages = -1):

    bs_last_page = BeautifulSoup(response.text, 'html.parser')
    v_last_page = int(bs_last_page.find('div', class_='pages').find('li', class_="page last_page").find('a').text)

    #O numero de paginas deve ser superior a -1 e menor/igual ao total de paginas
    if pages > -1 and pages <= v_last_page:
        v_last_page = pages
        
    return v_last_page

In [3]:

url_list = ["https://www.metacritic.com/game/playstation-4/the-last-of-us-remastered/user-reviews"
            , "https://www.metacritic.com/game/playstation-4/god-of-war-ragnarok/user-reviews"
            , "https://www.metacritic.com/game/pc/duke-nukem-forever/user-reviews"
            , "https://www.metacritic.com/game/playstation-4/biomutant/user-reviews"
            , "https://www.metacritic.com/game/pc/no-mans-sky/user-reviews"
            , "https://www.metacritic.com/game/playstation-4/cyberpunk-2077/user-reviews"]
user_agent = {'User-agent': 'Mozilla/5.0'}

In [4]:
review_lst = []

for url in url_list:
    response = requests.get(url, headers=user_agent)
    last_page = get_last_pages(url)
    title = get_title(url)
    page_num = 0

    print('-----Loading Title: ' + title)
    while page_num <= last_page:
        
        response = requests.get(url + "/user-reviews?page=" + str(page_num), headers=user_agent)

        #Scrap
        bs = BeautifulSoup(response.text, 'html.parser')
        reviews = bs.find_all('div', class_='review_content')
        review_lst.append(pd.DataFrame([{'title': title
                                        ,'name': get_name(row)
                                        ,'date' : get_date(row)
                                        ,'rating': get_rating(row)
                                        ,'comment': get_coment(row)
                                        } for row in reviews]))

        
        page_num = page_num+1
    print('-----Loaded Page: ' + str(page_num) + ' -------')
    
    df_review = pd.concat(review_lst)
    df_review = df_review[df_review['comment'].notna()]
    df_review['rating'] = df_review['rating'].astype("Int8")


-----Loading Title: The Last of Us Remastered
-----Loaded Page: 35 -------
-----Loading Title: God of War: Ragnarok
-----Loaded Page: 4 -------
-----Loading Title: Duke Nukem Forever
-----Loaded Page: 6 -------
-----Loading Title: Biomutant
-----Loaded Page: 4 -------
-----Loading Title: No Man's Sky
-----Loaded Page: 10 -------
-----Loading Title: Cyberpunk 2077
-----Loaded Page: 55 -------


In [5]:
df_review.head()

,title,name,date,rating,comment
1,The Last of Us Remastered,Nataraja,"Aug 1, 2014",10,"General Overview: Fantastic story, smooth grap..."
4,The Last of Us Remastered,brad0103triplex,"Jul 29, 2014",10,This is pretty much the same game I loved on t...
5,The Last of Us Remastered,awohlleb,"Jul 29, 2014",10,When you stand at the top of the heap with thi...
6,The Last of Us Remastered,FinalFantasy467,"Jul 30, 2014",10,The last of us is such an amazing experience a...
9,The Last of Us Remastered,MONG,"Aug 1, 2014",10,The Original Was Amazing – This One is Even Be...


In [6]:
df_review['title'].value_counts().sort_index()

Biomutant                     116
Cyberpunk 2077               1778
Duke Nukem Forever            282
God of War: Ragnarok           60
No Man's Sky                  398
The Last of Us Remastered     951
Name: title, dtype: int64

In [7]:
df_review.describe()

,rating
count,3585.0
mean,5.564296
std,4.009734
min,0.0
25%,1.0
50%,7.0
75%,10.0
max,10.0


In [8]:
df_review['rating'].value_counts().sort_index()

0     724
1     271
2     192
3     158
4     144
5     127
6     136
7     182
8     307
9     378
10    966
Name: rating, dtype: Int64

### Primeiros levantamentos (10/04/2023):
- Avaliações que não possuem comentarios foram removidos da base para evitar problemas com a analise de texto
- Amostra possui 3585 avaliações
- Média de avaliações em 5.5
- A mediana das avaliações = 7.5

## Preperação dos dados

In [9]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import spacy.cli


### Lematizer
- Criando a lista de stopwords e utilizando spaCy para lematização da lista


In [10]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

spacy.cli.download("en_core_web_sm")
nlp = spacy.load('en_core_web_sm')

stop_words_lemma = nlp(' '.join(stop_words))
stop_words_lemma = [' '.join(token.lemma_) for token in stop_words_lemma]


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\victo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [24]:

# Função para limpeza de texto
def preprocess_text(vtext):
    # remover pontuações
    vtext = vtext.translate(str.maketrans('', '', string.punctuation))
    vtext = vtext.lower()
    vtext = re.sub(r'[^a-zàáãâéêíóôõú\s]', '', vtext)

    return vtext

# Função para remover stopwords e aplica a lematização a um campo de texto
def text_lemmatizer(vtext):

    #limpando texto
    text_clear = preprocess_text(vtext)

    # Removendo stopwords
    token_list = text_clear.split()
    token_list = [token for token in token_list if token not in stop_words_lemma]

    # Juntando os tokens novamente em um único texto
    token_list = nlp(' '.join(token_list))
    token_list = [token.lemma_ for token in token_list]
    text = ' '.join(token_list)
    
    # text = text
    return text


In [25]:

df_review_token = df_review.copy()
df_review_token['comment'] = df_review_token['comment'].apply(text_lemmatizer)

# df_review_token

### Vetorizer
- Criação do vetor de caracteristica a partir da base criada anteriormente, já com aplicação da lematização e filtragem das stopwords

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer(
    binary= False,
    norm = 'l2',
    use_idf = True,
    token_pattern=r'\b[^\W\d]{3,}\b'
)

def vectorize(content, vectorizer: TfidfVectorizer = vectorizer):
    matrix = vectorizer.fit_transform(content)
    return matrix.toarray().tolist()

df_reviews_vectorized = df_review_token.copy()
df_reviews_vectorized["vectorizer"] = vectorize(df_reviews_vectorized["comment"])
df_reviews_vectorized['fx_rating'] = np.where(df_reviews_vectorized['rating'] <5, 'Negativo',
                                     np.where(df_reviews_vectorized['rating'] <7, 'Neutro',
                                     np.where(df_reviews_vectorized['rating'] <=10, 'Positivo', None)))

df_reviews_vectorized = df_reviews_vectorized[df_reviews_vectorized['fx_rating'].notna()]
# df_reviews_vectorized['vectorizer']

In [36]:
df_reviews_vectorized['fx_rating'].value_counts().sort_index()

Negativo    1489
Neutro       263
Positivo    1833
Name: fx_rating, dtype: int64

## Modelagem

In [53]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

X = df_reviews_vectorized["vectorizer"].tolist()
Y = df_reviews_vectorized['fx_rating']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=321)

model = MultinomialNB()
model = model.fit(X_train, y_train)

print("Acurácia do modelo: ", round(model.score(X_test, y_test), 2) * 100, "%")

Acurácia do modelo:  71.0 %


In [33]:
from sklearn import naive_bayes, tree

X = df_reviews_vectorized["vectorizer"].tolist()
Y = df_reviews_vectorized['fx_rating']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=123)

model = tree.DecisionTreeClassifier()
model = model.fit(X_train, y_train)

print("Acurácia do modelo: ", round(model.score(X_test, y_test), 2) * 100, "%")

Acurácia do modelo:  60.0 %


# Conclusão do trabalho
Apensar de um ser um modelo bem basico, foi possivel alcançar uma assertividade de 71% com o modelo multinomial, sendo um modelo utilizado para classificação.
Fiz um pequeno teste com o modelo da arvore de decisão porém é possivel ver que o resultado acabou sendo inferior.

Em determinado momento do trabalho ficou claro a importancia da utilização da lematização, para fazer o teste rodei o modelo apenas com limpeza dos dados e o resultado foi uma assertividade de 42%, -29 p.p. em relação ao modelo com a tokenzição.

Incluir outras variaveis, utilizar outras soluções para melhorar a tokenização e separar algumas avaliações que estão em outras linguas. São varias soluções que poderiam melhorar a acuracia do modelo. Um aprofundamento sobre a utilização de outros modelo e suas aplicações também parece ser bem interassante, pois facilita o entendimento e explicação do resultado e como consequencia nos leva a novas oportunidades de melhoria.